# Data Prepare

In [2]:
from collections import Counter
from itertools import combinations
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
import seaborn as sns

import pickle as pk

from jupyterthemes import jtplot
jtplot.style(figsize=(15, 9))

## Genres

### One-Hot Encoding

In [26]:
genres = pd.read_csv('../data/popular_10000_movies/movies_dataset_CLEAN.csv', index_col='id')

genres = genres['genres'].to_frame()
genres.dropna(inplace=True)

genres.head()

,genres
id,
385687,"Action, Crime, Thriller"
697843,"Action, Thriller"
603692,"Action, Thriller, Crime"
569094,"Action, Adventure, Animation, Science Fiction"
502356,"Animation, Family, Adventure, Fantasy, Comedy"


In [3]:
genres.shape

(9144, 1)

In [4]:
genres.isna().sum()

genres    0
dtype: int64

In [5]:
genres['genres'].nunique()

2131

In [4]:
unique_genres = [
    'Action',
    'Adventure',
    'Animation',
    'Comedy',
    'Crime',
    'Documentary',
    'Drama',
    'Family',
    'Fantasy',
    'History',
    'Horror',
    'Music',
    'Mystery',
    'Romance',
    'Science Fiction',
    'TV Movie',
    'Thriller',
    'War',
    'Western'
]

In [33]:
with open('data/unique_genres_list.pickle', 'wb') as f:
    pk.dump(unique_genres, f, pk.HIGHEST_PROTOCOL)

In [ ]:
genres_vectors = pd.DataFrame(index=genres.index, columns=unique_genres)
genres_vectors.head()

In [28]:
for i in range(genres.shape[0]):
    df = np.full(len(unique_genres), 0)
    
    for j in str(genres['genres'].iloc[i]).split(', '):
        df[unique_genres.index(j)] = 1

    genres_vectors.iloc[i] = df
        
genres_vectors.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,
385687,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
697843,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
603692,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
569094,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
502356,0,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0


### Labels for Genres

In [3]:
# if we have that genre in list it will be dominant, from most to least
labels = [
    'Documentary',
    'Music',
    'Animation',
    'Family',
    'Science Fiction',
    'Horror',
    'History',
    'Thriller',
]

In [30]:
labels_df = pd.DataFrame(index=genres.index, columns=['label'])
labels_df.head()

,label
id,
385687,NaN
697843,NaN
603692,NaN
569094,NaN
502356,NaN


In [31]:
for i in range(genres.shape[0]):
    genres_l = str(genres['genres'].iloc[i]).split(', ')
    labels_df.iloc[i] = genres_l[0]
    for j in labels:
        if j in genres_l:
            labels_df.iloc[i] = j
            break
        
labels_df.head()

,label
id,
385687,Thriller
697843,Thriller
603692,Thriller
569094,Animation
502356,Animation


In [32]:
le = LabelEncoder()
le.fit(labels_df['label'])

labels_df['label_encoder'] = le.transform(labels_df['label'])
labels_df.head()

,label,label_encoder
id,,
385687,Thriller,16
697843,Thriller,16
603692,Thriller,16
569094,Animation,2
502356,Animation,2


In [33]:
labels_df.shape

(9144, 2)

### Add Labels to Genres

In [34]:
df = pd.merge(genres_vectors, labels_df, on='id', how='inner')
df.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,...,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,label,label_encoder
id,,,,,,,,,,,,,,,,,,,,,
385687,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,Thriller,16
697843,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,Thriller,16
603692,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,Thriller,16
569094,1,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,Animation,2
502356,0,1,1,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,Animation,2


In [35]:
df.shape

(9144, 21)

In [37]:
df.to_csv('data/one-hot_genres_w_labels.csv')

### Cosine Similarity

In [43]:
genres_list = list(genres['genres'].unique())

In [44]:
for i, g in enumerate(genres_list):
    genres_list[i] = g.split(', ')

In [45]:
genres_list[:20]

[['Action', 'Crime', 'Thriller'],
 ['Action', 'Thriller'],
 ['Action', 'Thriller', 'Crime'],
 ['Action', 'Adventure', 'Animation', 'Science Fiction'],
 ['Animation', 'Family', 'Adventure', 'Fantasy', 'Comedy'],
 ['Action', 'Adventure', 'Science Fiction'],
 ['Animation', 'Comedy', 'Family', 'Fantasy', 'Romance'],
 ['Mystery', 'Thriller', 'Science Fiction'],
 ['Science Fiction', 'Action', 'Adventure'],
 ['Action', 'Science Fiction', 'Adventure'],
 ['Science Fiction', 'Adventure', 'Action'],
 ['Action', 'Horror', 'Thriller'],
 ['Adventure', 'Family', 'Fantasy', 'Romance'],
 ['Thriller', 'Horror'],
 ['Horror', 'Thriller'],
 ['War', 'Action', 'Thriller'],
 ['Drama', 'Romance'],
 ['Action', 'Fantasy', 'Thriller'],
 ['Comedy', 'Action', 'Fantasy'],
 ['Comedy', 'Drama']]

In [46]:
un_genres_list = [genre for genres in genres_list for genre in genres]

In [47]:
len(genres_list), len(un_genres_list)

(2131, 7856)

In [48]:
un_genres_list = list(set(un_genres_list))

In [49]:
len(un_genres_list)

19

In [50]:
un_genres_list

['TV Movie',
 'Thriller',
 'Action',
 'Mystery',
 'Crime',
 'Documentary',
 'Adventure',
 'Science Fiction',
 'Drama',
 'Animation',
 'Music',
 'Fantasy',
 'Family',
 'Romance',
 'Western',
 'War',
 'History',
 'Horror',
 'Comedy']

In [51]:
genres_list[:20]

[['Action', 'Crime', 'Thriller'],
 ['Action', 'Thriller'],
 ['Action', 'Thriller', 'Crime'],
 ['Action', 'Adventure', 'Animation', 'Science Fiction'],
 ['Animation', 'Family', 'Adventure', 'Fantasy', 'Comedy'],
 ['Action', 'Adventure', 'Science Fiction'],
 ['Animation', 'Comedy', 'Family', 'Fantasy', 'Romance'],
 ['Mystery', 'Thriller', 'Science Fiction'],
 ['Science Fiction', 'Action', 'Adventure'],
 ['Action', 'Science Fiction', 'Adventure'],
 ['Science Fiction', 'Adventure', 'Action'],
 ['Action', 'Horror', 'Thriller'],
 ['Adventure', 'Family', 'Fantasy', 'Romance'],
 ['Thriller', 'Horror'],
 ['Horror', 'Thriller'],
 ['War', 'Action', 'Thriller'],
 ['Drama', 'Romance'],
 ['Action', 'Fantasy', 'Thriller'],
 ['Comedy', 'Action', 'Fantasy'],
 ['Comedy', 'Drama']]

In [52]:
genres_counter = {}
l = []
for g in un_genres_list:
    for gs in genres_list:
        if g in gs:
            l.extend(gs)
    genres_counter[g] = dict(Counter(l))
    l = []

genres_counter['Animation']

{'Action': 129,
 'Adventure': 216,
 'Animation': 407,
 'Science Fiction': 99,
 'Family': 298,
 'Fantasy': 160,
 'Comedy': 215,
 'Romance': 26,
 'Music': 38,
 'Drama': 31,
 'TV Movie': 36,
 'Crime': 14,
 'Thriller': 12,
 'Horror': 18,
 'Mystery': 40,
 'Western': 7,
 'War': 5,
 'History': 4}

In [53]:
genres_df = pd.DataFrame(columns=un_genres_list, index=un_genres_list)
genres_df.head()

,TV Movie,Thriller,Action,Mystery,Crime,Documentary,Adventure,Science Fiction,Drama,Animation,Music,Fantasy,Family,Romance,Western,War,History,Horror,Comedy
TV Movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Thriller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Action,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mystery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Crime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
for genre in un_genres_list:
    g = {}
    for u_genre in un_genres_list:
        if u_genre in genres_counter[genre].keys():
            g[u_genre] =  genres_counter[genre][u_genre]
        else:
            g[u_genre] = 0
    genres_df.loc[genre] = g

In [55]:
genres_df.head()

,TV Movie,Thriller,Action,Mystery,Crime,Documentary,Adventure,Science Fiction,Drama,Animation,Music,Fantasy,Family,Romance,Western,War,History,Horror,Comedy
TV Movie,176,25,38,12,10,3,44,39,55,36,23,46,77,22,3,5,10,20,79
Thriller,25,615,264,196,197,1,111,154,285,12,4,73,8,72,19,38,38,159,98
Action,38,264,759,73,153,5,348,234,208,129,8,152,137,52,38,66,50,115,210
Mystery,12,196,73,342,98,3,59,72,141,40,5,52,41,36,6,6,9,84,85
Crime,10,197,153,98,348,3,43,25,166,14,9,15,15,39,13,2,19,36,116


In [56]:
cs_df = pd.DataFrame(columns=genres_df.index, index=genres_df.index)
cs_df.head()

,TV Movie,Thriller,Action,Mystery,Crime,Documentary,Adventure,Science Fiction,Drama,Animation,Music,Fantasy,Family,Romance,Western,War,History,Horror,Comedy
TV Movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Thriller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Action,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mystery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Crime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
def get_cosine_similarity_mertric(df1, df2):
    return np.dot(df1, df2)/ \
    (np.sqrt(np.power(df1, 2).sum()) * np.sqrt(np.power(df2, 2).sum()))

In [70]:
for i in genres_df.columns:
    for j in genres_df.columns:
        cs_df.loc[i][j] = get_cosine_similarity_mertric(genres_df[i], genres_df[j])

In [71]:
cs_df

,TV Movie,Thriller,Action,Mystery,Crime,Documentary,Adventure,Science Fiction,Drama,Animation,Music,Fantasy,Family,Romance,Western,War,History,Horror,Comedy
TV Movie,1.0,0.394733,0.513236,0.39673,0.364897,0.273702,0.592527,0.53523,0.511371,0.599006,0.580721,0.61596,0.676505,0.495483,0.354297,0.326719,0.332524,0.410583,0.677208
Thriller,0.394733,1.0,0.717784,0.803086,0.803562,0.252943,0.478655,0.632004,0.747659,0.276127,0.26195,0.419476,0.280956,0.495854,0.468874,0.527351,0.504018,0.717057,0.440748
Action,0.513236,0.717784,1.0,0.554684,0.650536,0.291962,0.797106,0.781199,0.623967,0.589238,0.364484,0.625936,0.571553,0.473524,0.555539,0.573059,0.479258,0.612385,0.641312
Mystery,0.39673,0.803086,0.554684,1.0,0.702164,0.263122,0.456686,0.554591,0.662259,0.365163,0.314863,0.449878,0.36879,0.471931,0.387748,0.379926,0.384377,0.646352,0.491783
Crime,0.364897,0.803562,0.650536,0.702164,1.0,0.289151,0.422055,0.450467,0.70113,0.281047,0.310511,0.34936,0.297321,0.481107,0.460968,0.420063,0.462907,0.507797,0.503363
Documentary,0.273702,0.252943,0.291962,0.263122,0.289151,1.0,0.247916,0.216827,0.337446,0.169529,0.257017,0.192158,0.198267,0.221919,0.214639,0.240606,0.312933,0.220365,0.274301
Adventure,0.592527,0.478655,0.797106,0.456686,0.422055,0.247916,1.0,0.719617,0.537524,0.801408,0.530574,0.794288,0.831916,0.530338,0.53405,0.451217,0.386237,0.464983,0.784823
Science Fiction,0.53523,0.632004,0.781199,0.554591,0.450467,0.216827,0.719617,1.0,0.512783,0.598825,0.354121,0.613756,0.575048,0.428716,0.385528,0.382743,0.295121,0.641385,0.601664
Drama,0.511371,0.747659,0.623967,0.662259,0.70113,0.337446,0.537524,0.512783,1.0,0.351065,0.497333,0.496137,0.432878,0.725442,0.603654,0.706889,0.748685,0.516213,0.542533
Animation,0.599006,0.276127,0.589238,0.365163,0.281047,0.169529,0.801408,0.598825,0.351065,1.0,0.590689,0.775147,0.926371,0.421453,0.339857,0.239284,0.19883,0.339585,0.789651


In [72]:
cs_df.to_csv('data/genres_cosine_similarity.csv')

---

## Keywords

In [2]:
keywords = pd.read_csv('../data/popular_10000_movies/movies_dataset_CLEAN.csv', index_col='id')
keywords = keywords['keywords'].to_frame()

In [3]:
keywords.head()

,keywords
id,
385687,"[{'id': 9663, 'name': 'sequel'}, {'id': 9748, ..."
697843,"[{'id': 3070, 'name': 'mercenary'}, {'id': 966..."
603692,"[{'id': 242, 'name': 'new york city'}, {'id': ..."
569094,"[{'id': 2858, 'name': 'sacrifice'}, {'id': 328..."
502356,"[{'id': 282, 'name': 'video game'}, {'id': 690..."


In [4]:
keywords.iloc[22]

keywords    [{'id': 160130, 'name': 'invisible person'}, {...
Name: 1131438, dtype: object

In [5]:
keywords.iloc[3398] = "[{'id': 5424, 'name': 'women's prison'}]"
keywords.iloc[8550] = "[{'id': 15101, 'name': 'based on children's book'}]"
keywords.iloc[9035] = "[{'id': 210450, 'name': 'noah's ark'}]"

In [6]:
keywords.iloc[2709].to_list()[0], keywords.iloc[4485].to_list()[0]

("[{'id': 18021, 'name': 'detroit, michigan'}]",
 "[{'id': 588, 'name': 'rome, italy'}]")

In [7]:
kw = keywords.iloc[2709].to_list()[0][1:-1]
l = re.findall("'name': '[\w()\-,\. ]+'", kw)
l2 = [i.replace("'name': ", "").replace("'", '') for i in l]
l, l2

(["'name': 'detroit, michigan'"], ['detroit, michigan'])

In [8]:
full_keywords_l = []
keywords_list_of_list = []
for i in range(keywords.shape[0]):
    kw = keywords.iloc[i].to_list()[0][1:-1]
    l = re.findall("'name': '[\w()\-,\. ]+'", kw)
    l2 = [i.replace("'name': ", "").replace("'", '') for i in l]
    if not l2:
        print(i)
    full_keywords_l.extend(l2)
    keywords_list_of_list.append(l2)

In [9]:
keywords_list_of_list[:5]

[['sequel', 'revenge', 'racing', 'family', 'cars'],
 ['mercenary',
  'sequel',
  'rescue mission',
  'long take',
  'based on graphic novel'],
 ['new york city',
  'martial arts',
  'hitman',
  'sequel',
  'organized crime',
  'osaka, japan',
  'aftercreditsstinger',
  'hunted',
  'professional assassin',
  'neo-noir',
  'berlin'],
 ['sacrifice',
  'villain',
  'comic book',
  'sequel',
  'superhero',
  'based on comic',
  'alternate dimension',
  'alternate version',
  'super power',
  'brooklyn, new york city',
  'superhero team',
  'spider bite',
  'super villain',
  'cliffhanger',
  'teen superhero',
  'alternate universe',
  'female superhero',
  'cartoon spider'],
 ['video game',
  'gorilla',
  'plumber',
  'magic mushroom',
  'anthropomorphism',
  'based on video game',
  'toad',
  'aftercreditsstinger',
  'duringcreditsstinger',
  'damsel in distress',
  'piano',
  'white gloves',
  'brother brother relationship',
  'evil king']]

In [160]:
keywords_list_of_str = [', '.join(l) for l in keywords_list_of_list] # bad idea with ', '
keywords_list_of_str[0]

'sequel, revenge, racing, family, cars'

In [10]:
len(full_keywords_l), len(keywords_list_of_list)

(82292, 9144)

In [11]:
unique_keywords = list(set(full_keywords_l))

In [22]:
for i in unique_keywords:
    if len(i)<3:
        print(i)

pc
gi
ax
ya
dj
بظ
3d
bp
iq
vr
rv
k2
살인
青春
uk


In [12]:
len(unique_keywords)

14239

In [24]:
keywords_counter = {}
l = []
for k in list(set(full_keywords_l)):
    for ks in keywords_list_of_list:
        if k in ks:
            l.extend(ks)
    keywords_counter[k] = dict(Counter(l))
    l = []

keywords_counter['sequel']

{'sequel': 457,
 'revenge': 31,
 'racing': 2,
 'family': 11,
 'cars': 1,
 'mercenary': 6,
 'rescue mission': 7,
 'long take': 1,
 'based on graphic novel': 1,
 'new york city': 12,
 'martial arts': 20,
 'hitman': 5,
 'organized crime': 2,
 'osaka, japan': 1,
 'aftercreditsstinger': 37,
 'hunted': 1,
 'professional assassin': 2,
 'neo-noir': 7,
 'berlin': 1,
 'sacrifice': 2,
 'villain': 10,
 'comic book': 1,
 'superhero': 31,
 'based on comic': 30,
 'alternate dimension': 5,
 'alternate version': 1,
 'super power': 16,
 'brooklyn, new york city': 2,
 'superhero team': 7,
 'spider bite': 1,
 'super villain': 2,
 'cliffhanger': 1,
 'teen superhero': 1,
 'alternate universe': 1,
 'female superhero': 1,
 'cartoon spider': 1,
 'hero': 7,
 'mad scientist': 4,
 'space opera': 5,
 'raccoon': 1,
 'duringcreditsstinger': 44,
 'marvel cinematic universe (mcu)': 12,
 'cosmic': 1,
 'outer space': 1,
 'chosen family': 1,
 'loss of loved one': 10,
 'dying and death': 3,
 'alien life-form': 4,
 'resurr

In [25]:
[] in keywords_list_of_list

False

In [50]:
with open('data/keywords_list_of_list.pickle', 'wb') as f:
    pk.dump(keywords_list_of_list, f, pk.HIGHEST_PROTOCOL)

In [51]:
with open('data/keywords_counter.pickle', 'wb') as f:
    pk.dump(keywords_counter, f, pk.HIGHEST_PROTOCOL)

In [28]:
df = pd.DataFrame(index=keywords.index, columns=['keywords_list'], data=np.array(keywords_list_of_list, dtype=object))
df.head()

,keywords_list
id,
385687,"[sequel, revenge, racing, family, cars]"
697843,"[mercenary, sequel, rescue mission, long take,..."
603692,"[new york city, martial arts, hitman, sequel, ..."
569094,"[sacrifice, villain, comic book, sequel, super..."
502356,"[video game, gorilla, plumber, magic mushroom,..."


In [29]:
df.shape

(9144, 1)

In [140]:
df.to_csv('data/keywords_list.csv')

In [142]:
df = pd.DataFrame(index=keywords.index, columns=['keywords_str'], data=np.array(keywords_list_of_str))
df.head()

,keywords_str
id,
385687,"sequel, revenge, racing, family, cars"
697843,"mercenary, sequel, rescue mission, long take, ..."
603692,"new york city, martial arts, hitman, sequel, o..."
569094,"sacrifice, villain, comic book, sequel, superh..."
502356,"video game, gorilla, plumber, magic mushroom, ..."


In [143]:
df.to_csv('data/keywords_str.csv')

In [2]:
keywords = pd.read_csv('data/keywords_list.csv', index_col='id')
keywords

,keywords_list
id,
385687,"['sequel', 'revenge', 'racing', 'family', 'cars']"
697843,"['mercenary', 'sequel', 'rescue mission', 'lon..."
603692,"['new york city', 'martial arts', 'hitman', 's..."
569094,"['sacrifice', 'villain', 'comic book', 'sequel..."
502356,"['video game', 'gorilla', 'plumber', 'magic mu..."
...,...
15017,"['stripper', 'friends', 'best friend']"
79509,"['new york city', 'detective', 'obsession', 'a..."
13370,"['mockumentary', 'folk singer']"


In [3]:
genres = pd.read_csv('data/unique_keywords_genres_count_with_labels.csv', index_col=0)
genres

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,label
car crash,20,10,1,16,11,0,15,4,5,0,8,1,6,3,9,1,25,0,0,"Thriller, Action, Comedy"
liposuction,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,"Comedy, Drama, Fantasy"
stasis,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,"Adventure, Mystery, Science Fiction"
giant vegetable,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,"Adventure, Animation, Comedy"
monster,48,43,22,41,0,0,17,27,50,0,71,1,12,0,55,8,29,1,0,"Horror, Science Fiction, Fantasy"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mad,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,"Comedy, Crime, Drama"
ophthalmologist,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,"Thriller, Action, Adventure"
gossip,0,0,0,3,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,"Comedy, Romance, Drama"
burial ground,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,"Drama, Horror, Western"


In [8]:
keywords_vectors_sum = pd.DataFrame(index=keywords.index, columns=genres.columns)
keywords_vectors_sum

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,label
id,,,,,,,,,,,,,,,,,,,,
385687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
697843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
569094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
keywords

,keywords_list
id,
385687,"['sequel', 'revenge', 'racing', 'family', 'cars']"
697843,"['mercenary', 'sequel', 'rescue mission', 'lon..."
603692,"['new york city', 'martial arts', 'hitman', 's..."
569094,"['sacrifice', 'villain', 'comic book', 'sequel..."
502356,"['video game', 'gorilla', 'plumber', 'magic mu..."
...,...
15017,"['stripper', 'friends', 'best friend']"
79509,"['new york city', 'detective', 'obsession', 'a..."
13370,"['mockumentary', 'folk singer']"


In [17]:
def vector_sum(row):
    _id = row.name
    _id_uk = genres.loc[keywords.loc[_id].to_list()[0][2:-2].split("', '")]
    return _id_uk.sum()

In [33]:
def cast_vector_sum(row):
    _id = row.name
    _id_uk = cast_genres.loc[cast.loc[_id].to_list()[0][2:-2].split("', '")]
    return _id_uk.sum()

In [19]:
keywords_vectors_sum = keywords.apply(vector_sum, axis=1)

In [21]:
keywords_vectors_sum.to_csv('data/keywords_vectors_sum.csv')

In [22]:
cast = pd.read_csv('data/cast_list.csv', index_col='id')
cast.head()

,cast_list
id,
385687,"['Vin Diesel', 'Michelle Rodriguez', 'Tyrese G..."
697843,"['Chris Hemsworth', 'Golshifteh Farahani', 'Ad..."
603692,"['Keanu Reeves', 'Donnie Yen', 'Bill Skarsgård..."
569094,"['Shameik Moore', 'Hailee Steinfeld', 'Brian T..."
502356,"['Chris Pratt', 'Anya Taylor-Joy', 'Charlie Da..."


In [32]:
cast_genres = pd.read_csv('data/unique_actors_genres_count.csv', index_col=0)
cast_genres.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
Sheldon Jett,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Jocelyn Ott,0,1,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0
Mel Archer,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
Tony Aitken,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
J.K. Simmons,24,24,12,31,9,2,30,12,9,4,3,3,2,9,12,1,14,0,3


In [34]:
cast_vectors_sum = pd.DataFrame(index=keywords.index, columns=genres.columns)
cast_vectors_sum = cast.apply(cast_vector_sum, axis=1)

In [35]:
cast_vectors_sum.to_csv('data/cast_vectors_sum.csv')

### Unique Keywords and Genres

In [18]:
uk = pd.DataFrame(0, index=unique_keywords, columns=[unique_genres])
uk.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
car crash,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
liposuction,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
stasis,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
giant vegetable,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
monster,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
kw_list = pd.read_csv('data/keywords_list.csv', index_col='id')
kw_list.head()

,keywords_list
id,
385687,"['sequel', 'revenge', 'racing', 'family', 'cars']"
697843,"['mercenary', 'sequel', 'rescue mission', 'lon..."
603692,"['new york city', 'martial arts', 'hitman', 's..."
569094,"['sacrifice', 'villain', 'comic book', 'sequel..."
502356,"['video game', 'gorilla', 'plumber', 'magic mu..."


In [21]:
genres.head()

,genres
id,
385687,"Action, Crime, Thriller"
697843,"Action, Thriller"
603692,"Action, Thriller, Crime"
569094,"Action, Adventure, Animation, Science Fiction"
502356,"Animation, Family, Adventure, Fantasy, Comedy"


In [36]:
kw_list = kw_list.merge(genres, on='id')

In [37]:
kw_list

,keywords_list,genres
id,,
385687,"['sequel', 'revenge', 'racing', 'family', 'cars']","Action, Crime, Thriller"
697843,"['mercenary', 'sequel', 'rescue mission', 'lon...","Action, Thriller"
603692,"['new york city', 'martial arts', 'hitman', 's...","Action, Thriller, Crime"
569094,"['sacrifice', 'villain', 'comic book', 'sequel...","Action, Adventure, Animation, Science Fiction"
502356,"['video game', 'gorilla', 'plumber', 'magic mu...","Animation, Family, Adventure, Fantasy, Comedy"
...,...,...
15017,"['stripper', 'friends', 'best friend']","Comedy, Romance"
79509,"['new york city', 'detective', 'obsession', 'a...","Action, Thriller, Comedy"
13370,"['mockumentary', 'folk singer']","Comedy, Music"


In [46]:
len(kw_list)

9144

In [47]:
for i in range(len(kw_list)):
    for j in kw_list.iloc[i]['keywords_list'][2:-2].split("', '"):
        for z in kw_list.iloc[i]['genres'].split(', '):
            uk.loc[j][z] += 1

In [48]:
uk

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
car crash,20,10,1,16,11,0,15,4,5,0,8,1,6,3,9,1,25,0,0
liposuction,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
stasis,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
giant vegetable,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
monster,48,43,22,41,0,0,17,27,50,0,71,1,12,0,55,8,29,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mad,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0
ophthalmologist,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
gossip,0,0,0,3,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0
burial ground,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1


In [ ]:
uk.to_csv('data/unique_keywords_genres_count.csv')

In [50]:
uk.loc['space']

Action             30
Adventure          33
Animation           7
Comedy              9
Crime               1
Documentary         1
Drama              14
Family             12
Fantasy            10
History             2
Horror              8
Music               0
Mystery             5
Romance             2
Science Fiction    59
TV Movie            4
Thriller           17
War                 0
Western             0
Name: space, dtype: int64

In [52]:
uk.describe()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
count,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000,14239.000000
mean,1.494276,1.119742,0.383946,1.802023,0.912775,0.062785,2.522930,0.656437,0.654049,0.314278,0.909755,0.182527,0.598708,0.910317,0.765012,0.091018,1.729265,0.200857,0.119671
std,6.525073,4.654708,2.513819,6.741369,4.572784,0.418348,9.902872,3.609652,3.151062,2.038377,4.353006,1.251284,2.720059,4.028732,4.093476,0.608737,7.000619,1.443207,0.849551
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000
max,226.000000,209.000000,91.000000,270.000000,223.000000,17.000000,631.000000,126.000000,124.000000,136.000000,207.000000,97.000000,155.000000,214.000000,183.000000,26.000000,368.000000,95.000000,39.000000


In [22]:
labels = [
    'Documentary',
    'Music',
    'Animation',
    'Family',
    'Science Fiction',
    'Horror',
    'History',
    'Thriller'
]

In [55]:
uk = pd.read_csv('data/unique_keywords_genres_count_with_labels.csv', index_col=0)
uk.drop('label', axis=1, inplace=True)
uk.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
car crash,20,10,1,16,11,0,15,4,5,0,8,1,6,3,9,1,25,0,0
liposuction,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
stasis,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
giant vegetable,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
monster,48,43,22,41,0,0,17,27,50,0,71,1,12,0,55,8,29,1,0


In [48]:
for i in uk.index.to_list():
    most_pop_genres = sorted(uk.loc[i].to_dict().items(), key=lambda x:x[1], reverse=True)
    uk.loc[i, 'label'] = ', '.join([x[0] for x in most_pop_genres][:3])

In [49]:
uk.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,label
car crash,20,10,1,16,11,0,15,4,5,0,8,1,6,3,9,1,25,0,0,"Thriller, Action, Comedy"
liposuction,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,"Comedy, Drama, Fantasy"
stasis,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,"Adventure, Mystery, Science Fiction"
giant vegetable,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,"Adventure, Animation, Comedy"
monster,48,43,22,41,0,0,17,27,50,0,71,1,12,0,55,8,29,1,0,"Horror, Science Fiction, Fantasy"


In [53]:
uk[uk['label']=='Adventure, Animation, Comedy']

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,label
giant vegetable,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,"Adventure, Animation, Comedy"
escape from jail,1,2,2,2,1,0,0,2,1,0,0,0,0,0,0,0,0,0,0,"Adventure, Animation, Comedy"
eyebrow,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,"Adventure, Animation, Comedy"
discworld,0,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,"Adventure, Animation, Comedy"
samba,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,"Adventure, Animation, Comedy"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
world series,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,"Adventure, Animation, Comedy"
sausage,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,"Adventure, Animation, Comedy"
blase,0,1,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,"Adventure, Animation, Comedy"
psychadelic,0,1,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,"Adventure, Animation, Comedy"


In [50]:
uk.to_csv('data/unique_keywords_genres_count_with_labels.csv')

In [2]:
uk = pd.read_csv('data/unique_keywords_genres_count_with_labels.csv', index_col=0)
uk

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,label
car crash,20,10,1,16,11,0,15,4,5,0,8,1,6,3,9,1,25,0,0,"Thriller, Action, Comedy"
liposuction,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,"Comedy, Drama, Fantasy"
stasis,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,"Adventure, Mystery, Science Fiction"
giant vegetable,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,"Adventure, Animation, Comedy"
monster,48,43,22,41,0,0,17,27,50,0,71,1,12,0,55,8,29,1,0,"Horror, Science Fiction, Fantasy"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mad,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,"Comedy, Crime, Drama"
ophthalmologist,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,"Thriller, Action, Adventure"
gossip,0,0,0,3,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,"Comedy, Romance, Drama"
burial ground,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,"Drama, Horror, Western"


In [3]:
uk.groupby('label').count()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
label,,,,,,,,,,,,,,,,,,,
"Action, Adventure, Animation",131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131,131
"Action, Adventure, Comedy",187,187,187,187,187,187,187,187,187,187,187,187,187,187,187,187,187,187,187
"Action, Adventure, Crime",40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40
"Action, Adventure, Documentary",9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
"Action, Adventure, Drama",199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Western, Comedy, Drama",2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
"Western, Drama, Action",7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
"Western, Drama, Adventure",3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


In [10]:
for i in uk.index.to_list():
    for j in uk.loc[i, 'label'].split(', '):
        if j in labels:
            uk.loc[i, 'label'] = j
            break

In [11]:
uk.groupby('label').count()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
label,,,,,,,,,,,,,,,,,,,
"Action, Adventure, Comedy",187,187,187,187,187,187,187,187,187,187,187,187,187,187,187,187,187,187,187
"Action, Adventure, Crime",40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40
"Action, Adventure, Drama",199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199
"Action, Adventure, Fantasy",164,164,164,164,164,164,164,164,164,164,164,164,164,164,164,164,164,164,164
"Action, Adventure, Mystery",8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Western, Comedy, Adventure",2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
"Western, Comedy, Drama",2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
"Western, Drama, Action",7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7


In [19]:
uk.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,label
car crash,20,10,1,16,11,0,15,4,5,0,8,1,6,3,9,1,25,0,0,Thriller
liposuction,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,"Comedy, Drama, Fantasy"
stasis,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,Science Fiction
giant vegetable,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,Animation
monster,48,43,22,41,0,0,17,27,50,0,71,1,12,0,55,8,29,1,0,Horror


In [12]:
uk.to_csv('data/unique_keywords_genres_count_with_labels_SHORTED_VERSION.csv')

In [ ]:
cs = cosine_similarity(uk.to_numpy(), uk.to_numpy())

In [4]:
uk_cos_sim = pd.DataFrame(index=uk.index, columns=uk.index, data=cs)
uk_cos_sim.shape

(14239, 14239)

In [14]:
# uk_cos_sim.to_csv('keywords_cosine_similarity.csv') ==> 2.5GB ))))

### Label Encoding

In [2]:
uk_le = pd.read_csv('data/unique_keywords_genres_count_with_labels_SHORTED_VERSION.csv', index_col=0)
uk_le.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,label
car crash,20,10,1,16,11,0,15,4,5,0,8,1,6,3,9,1,25,0,0,Thriller
liposuction,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,"Comedy, Drama, Fantasy"
stasis,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,Science Fiction
giant vegetable,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,Animation
monster,48,43,22,41,0,0,17,27,50,0,71,1,12,0,55,8,29,1,0,Horror


In [3]:
le = LabelEncoder()
le.fit(uk_le['label'])

uk_le['label_encoder'] = le.transform(uk_le['label'])
uk_le.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,...,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,label,label_encoder
car crash,20,10,1,16,11,0,15,4,5,0,...,1,6,3,9,1,25,0,0,Thriller,233
liposuction,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,"Comedy, Drama, Fantasy",82
stasis,0,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,Science Fiction,231
giant vegetable,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,Animation,63
monster,48,43,22,41,0,0,17,27,50,0,...,1,12,0,55,8,29,1,0,Horror,204


In [6]:
uk_le.to_csv('data/unique_keywords_genres_count_with_labels_SHORTED_VERSION_label_encoder.csv')

### One-Hot Labeling

In [31]:
keywords_ind = pd.read_csv('data/keywords_list.csv', index_col='id')
kw_vectors = pd.DataFrame(index=keywords_ind.index, columns=unique_keywords)
kw_vectors.shape

(9144, 14239)

In [32]:
keywords_ind.head()

,keywords_list
id,
385687,"['sequel', 'revenge', 'racing', 'family', 'cars']"
697843,"['mercenary', 'sequel', 'rescue mission', 'lon..."
603692,"['new york city', 'martial arts', 'hitman', 's..."
569094,"['sacrifice', 'villain', 'comic book', 'sequel..."
502356,"['video game', 'gorilla', 'plumber', 'magic mu..."


In [146]:
keywords_ind.isna().sum()

keywords_str    0
dtype: int64

In [30]:
'osaka, japan' in unique_keywords

True

In [41]:
keywords_ind['keywords_list'].iloc[2][2:-2].split("', '")

['new york city',
 'martial arts',
 'hitman',
 'sequel',
 'organized crime',
 'osaka, japan',
 'aftercreditsstinger',
 'hunted',
 'professional assassin',
 'neo-noir',
 'berlin']

In [42]:
len_kw_v = len(unique_keywords)

for i in range(kw_vectors.shape[0]):
    df = np.array(np.array([0]*len_kw_v))
    
    for j in keywords_ind['keywords_list'].iloc[i][2:-2].split("', '"):
        df[unique_keywords.index(j)] = 1

    kw_vectors.iloc[i] = df
        
kw_vectors.iloc[0:10, 0:10]

,sea world,expulsion,storm at sea,one-sided love,mobile home,muscular dystrophy,street art,undead,classic,rude male lead
id,,,,,,,,,,
385687,0,0,0,0,0,0,0,0,0,0
697843,0,0,0,0,0,0,0,0,0,0
603692,0,0,0,0,0,0,0,0,0,0
569094,0,0,0,0,0,0,0,0,0,0
502356,0,0,0,0,0,0,0,0,0,0
667538,0,0,0,0,0,0,0,0,0,0
976573,0,0,0,0,0,0,0,0,0,0
536437,0,0,0,0,0,0,0,0,0,0
298618,0,0,0,0,0,0,0,0,0,0


In [47]:
kw_vectors['martial arts'].sum()

200

In [48]:
kw_vectors.to_csv('data/one-hot_keywords.csv')

---

## Cast

In [6]:
cast = pd.read_csv('../data/popular_10000_movies/movies_dataset_CLEAN.csv', index_col='id')
cast.head()

,genre_ids,overview,popularity,release_date,title,vote_average,vote_count,genres,cast,crew,keywords
id,,,,,,,,,,,
385687,"[28, 80, 53]",Over many missions and against impossible odds...,4654.279,2023-05-17,Fast X,7.3,2093,"Action, Crime, Thriller","[{'cast_id': 12835, 'name': 'Vin Diesel', 'cha...","[{'crew_id': 1302, 'name': 'Susie Figgis', 'de...","[{'id': 9663, 'name': 'sequel'}, {'id': 9748, ..."
697843,"[28, 53]",Tasked with extracting a family who is at the ...,2494.010,2023-06-09,Extraction 2,7.7,910,"Action, Thriller","[{'cast_id': 74568, 'name': 'Chris Hemsworth',...","[{'crew_id': 950, 'name': 'Pietro Scalia', 'de...","[{'id': 3070, 'name': 'mercenary'}, {'id': 966..."
603692,"[28, 53, 80]","With the price on his head ever increasing, Jo...",1920.127,2023-03-22,John Wick: Chapter 4,7.9,3344,"Action, Thriller, Crime","[{'cast_id': 6384, 'name': 'Keanu Reeves', 'ch...","[{'crew_id': 3615, 'name': 'Manfred Banach', '...","[{'id': 242, 'name': 'new york city'}, {'id': ..."
569094,"[28, 12, 16, 878]","After reuniting with Gwen Stacy, Brooklyn’s fu...",2013.795,2023-05-31,Spider-Man: Across the Spider-Verse,8.6,1796,"Action, Adventure, Animation, Science Fiction","[{'cast_id': 587506, 'name': 'Shameik Moore', ...","[{'crew_id': 7624, 'name': 'Stan Lee', 'depart...","[{'id': 2858, 'name': 'sacrifice'}, {'id': 328..."
502356,"[16, 10751, 12, 14, 35]","While working underground to fix a water main,...",1539.037,2023-04-05,The Super Mario Bros. Movie,7.8,5165,"Animation, Family, Adventure, Fantasy, Comedy","[{'cast_id': 73457, 'name': 'Chris Pratt', 'ch...","[{'crew_id': 70851, 'name': 'Jack Black', 'dep...","[{'id': 282, 'name': 'video game'}, {'id': 690..."


In [7]:
cast = cast['cast'].to_frame()
cast.head()

,cast
id,
385687,"[{'cast_id': 12835, 'name': 'Vin Diesel', 'cha..."
697843,"[{'cast_id': 74568, 'name': 'Chris Hemsworth',..."
603692,"[{'cast_id': 6384, 'name': 'Keanu Reeves', 'ch..."
569094,"[{'cast_id': 587506, 'name': 'Shameik Moore', ..."
502356,"[{'cast_id': 73457, 'name': 'Chris Pratt', 'ch..."


In [8]:
cast.drop(cast[cast['cast']=='[]'].index, inplace=True)

In [9]:
cast[cast['cast']=='[]']

,cast
id,


In [10]:
cast.iloc[978].to_list()

['[{\'cast_id\': 94421, \'name\': \'Bobby Campo\', \'character\': "Nick O\'Bannon"}, {\'cast_id\': 94423, \'name\': \'Shantel VanSanten\', \'character\': \'Lori Milligan\'}, {\'cast_id\': 34, \'name\': \'Mykelti Williamson\', \'character\': \'George Lanter\'}, {\'cast_id\': 84076, \'name\': \'Nick Zano\', \'character\': \'Hunt Wynorski\'}, {\'cast_id\': 94424, \'name\': \'Haley Webb\', \'character\': \'Janet Cunningham\'}, {\'cast_id\': 21858, \'name\': \'Krista Allen\', \'character\': \'Samantha Lane\'}, {\'cast_id\': 94427, \'name\': \'Stephanie Honoré\', \'character\': "Mechanic\'s Girlfriend"}, {\'cast_id\': 65699, \'name\': \'Andrew Fiscella\', \'character\': \'Mechanic\'}, {\'cast_id\': 74136, \'name\': \'Justin Welborn\', \'character\': \'Racist\'}, {\'cast_id\': 94428, \'name\': \'Lara Grice\', \'character\': "Racist\'s Wife"}, {\'cast_id\': 94429, \'name\': \'Jackson Walker\', \'character\': \'Cowboy\'}, {\'cast_id\': 21249, \'name\': \'Jedda Jones\', \'character\': \'Nurse\'}

In [11]:
kw = cast.iloc[0].to_list()[0][1:-1]
l = re.findall("'name': '[\w()\-,\. ]+'", kw)
l2 = [i.replace("'name': ", "").replace("'", '') for i in l]
l2

['Vin Diesel',
 'Michelle Rodriguez',
 'Tyrese Gibson',
 'Ludacris',
 'John Cena',
 'Nathalie Emmanuel',
 'Jordana Brewster',
 'Sung Kang',
 'Jason Momoa',
 'Scott Eastwood',
 'Daniela Melchior',
 'Alan Ritchson',
 'Helen Mirren',
 'Brie Larson',
 'Jason Statham',
 'Charlize Theron',
 'Rita Moreno',
 'Joaquim de Almeida',
 'Leo A. Perry',
 'Luis Da Silva Jr.',
 'Jaz Hutchins',
 'Luka Hays',
 'Alexander Capon',
 'Pete Davidson',
 'Shadrach Agozino',
 'Ludmilla',
 'Miraj Grbić',
 'Meadow Walker Thornton-Allan',
 'Michael Irby',
 'Shahir Figueira',
 'Ben-Hur Santos',
 'Debby Ryan',
 'Josh Dun',
 'Robert Bastens',
 'Dwayne Johnson',
 'Gal Gadot']

In [25]:
full_cast_l = []
cast_list_of_list = []
for i in range(cast.shape[0]):
    kw = cast.iloc[i].to_list()[0][1:-1]
    l = re.findall("'name': '[\w()\-,\. ]+'", kw)
    l2 = [i.replace("'name': ", "").replace("'", '') for i in l]
    if not l2:
        print(i)
    full_cast_l.extend(l2)
    cast_list_of_list.append(l2)

In [26]:
len(full_cast_l), len(cast_list_of_list)

(311707, 9133)

In [27]:
unique_actors = list(set(full_cast_l))
len(unique_actors)

145215

In [ ]:
actors_counter = {}
l = []
for k in list(set(full_cast_l)):
    for ks in cast_list_of_list:
        if k in ks:
            l.extend(ks)
    actors_counter[k] = dict(Counter(l))
    l = []

actors_counter['Jordana Brewster']

In [32]:
with open('data/cast_list_of_list.pickle', 'wb') as f:
    pk.dump(cast_list_of_list, f, pk.HIGHEST_PROTOCOL)

In [31]:
with open('data/actors_counter.pickle', 'wb') as f:
    pk.dump(actors_counter, f, pk.HIGHEST_PROTOCOL)

In [35]:
df = pd.DataFrame(index=cast.index, columns=['cast_list'], data=np.array(cast_list_of_list, dtype=object))

In [37]:
df.to_csv('data/cast_list.csv')

In [9]:
cast_list = pd.read_csv('data/cast_list.csv', index_col='id')
cast_list.head()

,cast_list
id,
385687,"['Vin Diesel', 'Michelle Rodriguez', 'Tyrese G..."
697843,"['Chris Hemsworth', 'Golshifteh Farahani', 'Ad..."
603692,"['Keanu Reeves', 'Donnie Yen', 'Bill Skarsgård..."
569094,"['Shameik Moore', 'Hailee Steinfeld', 'Brian T..."
502356,"['Chris Pratt', 'Anya Taylor-Joy', 'Charlie Da..."


In [10]:
with open('data/cast_list_of_list.pickle', 'rb') as f:
    cast_list_of_list = pk.load(f)

In [11]:
cast_list_of_list[:10]

[['Vin Diesel',
  'Michelle Rodriguez',
  'Tyrese Gibson',
  'Ludacris',
  'John Cena',
  'Nathalie Emmanuel',
  'Jordana Brewster',
  'Sung Kang',
  'Jason Momoa',
  'Scott Eastwood',
  'Daniela Melchior',
  'Alan Ritchson',
  'Helen Mirren',
  'Brie Larson',
  'Jason Statham',
  'Charlize Theron',
  'Rita Moreno',
  'Joaquim de Almeida',
  'Leo A. Perry',
  'Luis Da Silva Jr.',
  'Jaz Hutchins',
  'Luka Hays',
  'Alexander Capon',
  'Pete Davidson',
  'Shadrach Agozino',
  'Ludmilla',
  'Miraj Grbić',
  'Meadow Walker Thornton-Allan',
  'Michael Irby',
  'Shahir Figueira',
  'Ben-Hur Santos',
  'Debby Ryan',
  'Josh Dun',
  'Robert Bastens',
  'Dwayne Johnson',
  'Gal Gadot'],
 ['Chris Hemsworth',
  'Golshifteh Farahani',
  'Adam Bessa',
  'Tornike Gogrichiani',
  'Tornike Bziava',
  'Tinatin Dalakishvili',
  'Andro Japaridze',
  'Justin Howell',
  'Idris Elba',
  'Olga Kurylenko',
  'Sinéad Phelps',
  'Patrick Newall',
  'Daniel Bernhardt',
  'Levan Saginashvili',
  'George Lasha',


In [12]:
unique_actors = [actor for cast in cast_list_of_list for actor in cast]

In [13]:
unique_actors = list(set(unique_actors))

In [14]:
len(unique_actors)

145215

In [16]:
with open('data/unique_cast_list.pickle', 'wb') as f:
    pk.dump(unique_actors, f, pk.HIGHEST_PROTOCOL)

In [24]:
ua = pd.DataFrame(0, index=unique_actors, columns=[unique_genres])
ua.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
Sheldon Jett,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Jocelyn Ott,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Mel Archer,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Tony Aitken,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
J.K. Simmons,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
cast_list = cast_list.merge(genres, on='id')
cast_list.head()

,cast_list,genres
id,,
385687,"['Vin Diesel', 'Michelle Rodriguez', 'Tyrese G...","Action, Crime, Thriller"
697843,"['Chris Hemsworth', 'Golshifteh Farahani', 'Ad...","Action, Thriller"
603692,"['Keanu Reeves', 'Donnie Yen', 'Bill Skarsgård...","Action, Thriller, Crime"
569094,"['Shameik Moore', 'Hailee Steinfeld', 'Brian T...","Action, Adventure, Animation, Science Fiction"
502356,"['Chris Pratt', 'Anya Taylor-Joy', 'Charlie Da...","Animation, Family, Adventure, Fantasy, Comedy"


In [28]:
for i in range(len(cast_list)):
    for j in cast_list.iloc[i]['cast_list'][2:-2].split("', '"):
        for z in cast_list.iloc[i]['genres'].split(', '):
            ua.loc[j][z] += 1

In [54]:
ua.to_csv('data/unique_actors_genres_count.csv')

In [30]:
ua.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
Sheldon Jett,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Jocelyn Ott,0,1,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0
Mel Archer,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
Tony Aitken,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
J.K. Simmons,24,24,12,31,9,2,30,12,9,4,3,3,2,9,12,1,14,0,3


In [ ]:
for i in ua.index.to_list():
    most_pop_genres = sorted(ua.loc[i].to_dict().items(), key=lambda x:x[1], reverse=True)
    ua.loc[i, 'label'] = ', '.join([x[0] for x in most_pop_genres][:3])

In [ ]:
ua.to_csv('data/unique_actors_genres_count_with_labels.csv')

In [ ]:
for i in ua.index.to_list():
    for j in ua.loc[i, 'label'].split(', '):
        if j in labels:
            ua.loc[i, 'label'] = j
            break

In [ ]:
ua.to_csv('data/unique_actors_genres_count_with_labels_SHORTED_VERSION.csv')

### Label Encoding

In [7]:
ua_le = pd.read_csv('data/unique_actors_genres_count_with_labels_SHORTED_VERSION.csv', index_col=0)
ua_le.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,label
Sheldon Jett,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,"Comedy, Romance, Action"
Jocelyn Ott,0,1,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,"Adventure, Comedy, Drama"
Mel Archer,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,"Drama, Western, Action"
Tony Aitken,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,"Drama, Romance, Action"
J.K. Simmons,24,24,12,31,9,2,30,12,9,4,3,3,2,9,12,1,14,0,3,"Comedy, Drama, Action"


In [8]:
le = LabelEncoder()
le.fit(ua_le['label'])

ua_le['label_encoder'] = le.transform(ua_le['label'])
ua_le.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,...,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,label,label_encoder
Sheldon Jett,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,"Comedy, Romance, Action",138
Jocelyn Ott,0,1,0,1,0,0,1,1,1,0,...,0,0,1,0,0,0,0,0,"Adventure, Comedy, Drama",61
Mel Archer,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,"Drama, Western, Action",253
Tony Aitken,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,"Drama, Romance, Action",234
J.K. Simmons,24,24,12,31,9,2,30,12,9,4,...,3,2,9,12,1,14,0,3,"Comedy, Drama, Action",117


In [11]:
ua_le['label_encoder'].max(), ua_le.shape

(359, (145215, 21))

In [12]:
ua_le.to_csv('data/unique_actors_genres_count_with_labels_SHORTED_VERSION_label_encoder.csv')

## Crew

In [2]:
crew = pd.read_csv('../data/popular_10000_movies/movies_dataset_CLEAN.csv', index_col='id')
crew = crew['crew'].to_frame()
crew.head()

,crew
id,
385687,"[{'crew_id': 1302, 'name': 'Susie Figgis', 'de..."
697843,"[{'crew_id': 950, 'name': 'Pietro Scalia', 'de..."
603692,"[{'crew_id': 3615, 'name': 'Manfred Banach', '..."
569094,"[{'crew_id': 7624, 'name': 'Stan Lee', 'depart..."
502356,"[{'crew_id': 70851, 'name': 'Jack Black', 'dep..."


In [53]:
crew.drop(crew[crew['crew']=='[]'].index, inplace=True)

In [ ]:
# production, directing, writing

In [8]:
crew.loc[385687].to_list()

['[{\'crew_id\': 1302, \'name\': \'Susie Figgis\', \'department\': \'Production\'}, {\'crew_id\': 2519, \'name\': \'Sanja Milković Hays\', \'department\': \'Costume & Make-Up\'}, {\'crew_id\': 7239, \'name\': \'Peter Brown\', \'department\': \'Sound\'}, {\'crew_id\': 7239, \'name\': \'Peter Brown\', \'department\': \'Sound\'}, {\'crew_id\': 10570, \'name\': \'Joseph M. Caracciolo Jr.\', \'department\': \'Production\'}, {\'crew_id\': 10570, \'name\': \'Joseph M. Caracciolo Jr.\', \'department\': \'Production\'}, {\'crew_id\': 6041, \'name\': \'Brian Tyler\', \'department\': \'Sound\'}, {\'crew_id\': 8162, \'name\': \'Gary Scott Thompson\', \'department\': \'Writing\'}, {\'crew_id\': 11874, \'name\': \'Neal H. Moritz\', \'department\': \'Production\'}, {\'crew_id\': 13162, \'name\': \'Dov Samuel\', \'department\': \'Editing\'}, {\'crew_id\': 12835, \'name\': \'Vin Diesel\', \'department\': \'Production\'}, {\'crew_id\': 15227, \'name\': \'Sharon Smith Holley\', \'department\': \'Visual E

In [44]:
kw = crew.iloc[0].to_list()[0][1:-1]
l = re.findall("'name': '[\w()\-,\. ]+', 'department': 'Production'|'name': '[\w()\-,\. ]+', 'department': 'Writing'|'name': '[\w()\-,\. ]+', 'department': 'Directing'", kw)
l2 = [i.replace("'name': ", "").replace("'", '').split(', ')[0] for i in l]
list(set(l2))

['Zach Dean',
 'Amanda Lewis',
 'Gary Scott Thompson',
 'Luís Sérgio',
 'Dan Mazeau',
 'Jeff Kirschenbaum',
 'Katie Byles',
 'Vincent Lascoumes',
 'Danielle Stanners',
 'Alexander Witt',
 'Eleri Coulten',
 'Spiro Razatos',
 'Neal H. Moritz',
 'Joshua Benedetti',
 'Kirsty Kinnear',
 'Chris Morgan',
 'Samantha Vincent',
 'Kyle Williams',
 'David Cain',
 'Louis Leterrier',
 'Justin Lin',
 'Matt Craufurd',
 'Samantha Arnold',
 'Vin Diesel',
 'Matt Risebrow',
 'Joseph M. Caracciolo Jr.',
 'Olivier Schneider',
 'Danny Khoudary',
 'Nick White',
 'Lex Donovan',
 'Mark Bomback',
 'Susie Figgis',
 'Zoe Morgan']

In [73]:
def prepare_crew(row):
    kw = row[1:-1]
    l = re.findall("'name': '[\w()\-,\. ]+', 'department': 'Production'|'name': '[\w()\-,\. ]+', 'department': 'Writing'|'name': '[\w()\-,\. ]+', 'department': 'Directing'", kw)
    l2 = [i.replace("'name': ", "").replace("'", '').split(', ')[0] for i in l]
    return list(set(l2))

In [74]:
crew['crew_'] = crew['crew'].apply(prepare_crew)

In [77]:
crew['crew_'].to_csv('data/crew_list.csv')

### Unique

In [3]:
crew = pd.read_csv('data/crew_list.csv', index_col='id')
crew.head()

,crew_
id,
385687,"['Zach Dean', 'Amanda Lewis', 'Gary Scott Thom..."
697843,"['Mike Larocca', 'Stephen McFeely', 'Joe Russo..."
603692,"['Michael Finch', 'Louise Rosner-Meyer', 'Henn..."
569094,"['Griffin Johnston', 'Phil Lord', 'Robert Jonk..."
502356,"['David D. Au', 'Michael Jelenic', 'Matt Fogel..."


In [4]:
crew.iloc[0].to_list()[0][2:-2].split("', '")

['Zach Dean',
 'Amanda Lewis',
 'Gary Scott Thompson',
 'Luís Sérgio',
 'Dan Mazeau',
 'Jeff Kirschenbaum',
 'Katie Byles',
 'Vincent Lascoumes',
 'Danielle Stanners',
 'Alexander Witt',
 'Eleri Coulten',
 'Spiro Razatos',
 'Neal H. Moritz',
 'Joshua Benedetti',
 'Kirsty Kinnear',
 'Chris Morgan',
 'Samantha Vincent',
 'Kyle Williams',
 'David Cain',
 'Louis Leterrier',
 'Justin Lin',
 'Matt Craufurd',
 'Samantha Arnold',
 'Vin Diesel',
 'Matt Risebrow',
 'Joseph M. Caracciolo Jr.',
 'Olivier Schneider',
 'Danny Khoudary',
 'Nick White',
 'Lex Donovan',
 'Mark Bomback',
 'Susie Figgis',
 'Zoe Morgan']

In [5]:
unique_crew = []
for i in range(crew.shape[0]):
    unique_crew.extend(crew.iloc[i].to_list()[0][2: -2].split("', '"))

unique_crew = list(set(unique_crew))

In [6]:
unique_crew[:10]

['Harmon Kaslow',
 'Gene Patterson',
 'Shirel Kozak',
 'Paul Freiberger',
 'Alex Theurer',
 'David Fountain',
 'Jean Skinner',
 'Gerardo Gatica',
 'Rich Thorne',
 'Iddo Lampton Enochs Jr.']

In [7]:
len(unique_crew)

54769

In [8]:
with open('data/unique_crew_list.pickle', 'wb') as f:
    pk.dump(unique_crew, f, pk.HIGHEST_PROTOCOL)

In [32]:
unique_genres

['Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western']

In [77]:
crew_vectors = pd.DataFrame(0, index=unique_crew, columns=unique_genres)
crew_vectors.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
Bob Ogle,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Michael Mack,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Stuart Hoole,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Suzanne Cabot,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Anastacia C. Nemec,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [108]:
oh_genres = pd.read_csv('data/one-hot_genres.csv', index_col='id')
oh_genres.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,
385687,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
697843,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
603692,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
569094,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
502356,0,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0


In [106]:
oh_genres = oh_genres.loc[crew.index]

In [117]:
oh_genres = oh_genres[~oh_genres.index.duplicated(keep='first')]

In [120]:
for _id in crew.index:
    team = crew.loc[_id].to_list()[0][2: -2].split("', '")
    genres_vector = oh_genres.loc[_id].to_list()
    for person in team:
        crew_vectors.loc[person] += genres_vector
    

In [122]:
crew_vectors.to_csv('data/crew_vectors.csv')

In [7]:
crew_vectors = pd.read_csv('data/crew_vectors.csv', index_col=0)
crew_vectors.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
Bob Ogle,0,0,3,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0
Michael Mack,0,0,3,3,0,0,0,3,3,0,0,0,0,0,0,0,0,0,0
Stuart Hoole,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0
Suzanne Cabot,0,0,0,6,0,0,0,0,6,0,0,0,6,6,0,0,9,0,0
Anastacia C. Nemec,6,0,0,3,6,0,3,0,0,0,6,0,6,3,0,0,6,0,0


In [12]:
def vector_sum(row):
    _id = row.name
    _id_uc = crew_vectors.loc[crew.loc[_id].to_list()[0][2:-2].split("', '")]
    return _id_uc.sum()

In [6]:
crew_vectors_sum = pd.DataFrame(index=crew.index, columns=unique_genres)
crew_vectors_sum.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,
385687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
697843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
569094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
crew_vectors_sum = crew.apply(vector_sum, axis=1)

In [14]:
crew_vectors_sum.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,
385687,1197,621,9,345,612,3,447,168,321,75,135,24,87,153,312,0,786,39,3
697843,843,477,12,147,123,0,195,78,174,15,51,0,33,51,375,0,423,9,6
603692,930,414,15,342,462,3,348,18,159,51,144,12,153,120,273,0,816,66,9
569094,1635,1506,1086,585,126,6,262,465,540,37,117,27,78,63,1134,27,261,18,6
502356,69,390,573,540,24,0,12,540,276,0,0,33,3,0,27,3,0,0,0


In [15]:
crew_vectors_sum.to_csv('data/crew_vectors_sum.csv')